In [1]:
import os
import shutil
import random

def split_dataset(source_dir, dest_dir, train_ratio=0.7, valid_ratio=0.15, test_ratio=0.15):
    """
    Splits a classification dataset into train, valid, and test sets.
    Ensures that each split contains at least one image per class.
    """
    if not (train_ratio + valid_ratio + test_ratio) == 1.0:
        raise ValueError("Train, valid, and test ratios must sum to 1.0")

    print("Starting dataset preparation...")
    print(f"Source Directory: {source_dir}")
    print(f"Destination Directory: {dest_dir}")

    # Step 1: Find all class names by looking inside 'Train' and 'Validation' subdirectories
    all_class_names = set()
    for split_folder in ['Train', 'Validation']:
        split_path = os.path.join(source_dir, split_folder)
        if os.path.exists(split_path):
            class_names_in_split = [d for d in os.listdir(split_path) if os.path.isdir(os.path.join(split_path, d))]
            all_class_names.update(class_names_in_split)
    
    class_names = sorted(list(all_class_names))
    if not class_names:
        print("No class directories found inside 'Train' or 'Validation'. Please check your source path.")
        return

    print(f"Found {len(class_names)} classes: {class_names}")
    
    # Step 2: Create destination directories
    for split in ['train', 'valid', 'test']:
        for class_name in class_names:
            os.makedirs(os.path.join(dest_dir, split, class_name), exist_ok=True)

    print("Created new directory structure.")

    # Step 3: Collect all images for each class from both source directories
    class_images = {class_name: [] for class_name in class_names}
    for split_folder in ['Train', 'Validation']:
        split_path = os.path.join(source_dir, split_folder)
        if os.path.exists(split_path):
            for class_name in class_names:
                class_path = os.path.join(split_path, class_name)
                if os.path.exists(class_path):
                    # Collect all image files, ignoring subdirectories
                    images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
                    class_images[class_name].extend([os.path.join(class_path, img) for img in images])

    # Step 4: Split and copy files
    for class_name, all_images in class_images.items():
        if not all_images:
            print(f"Warning: No images found for class '{class_name}'. Skipping.")
            continue

        random.shuffle(all_images)
        total_images = len(all_images)
        print(f"\nProcessing class: {class_name}")
        print(f"Total images: {total_images}")

        train_end = int(total_images * train_ratio)
        valid_end = train_end + int(total_images * valid_ratio)

        train_set = all_images[:train_end]
        valid_set = all_images[train_end:valid_end]
        test_set = all_images[valid_end:]

        # Ensure at least one image in each split for all classes
        if not valid_set and total_images > 1:
            valid_set = [train_set.pop()]
        if not test_set and total_images > 2:
            test_set = [train_set.pop()]

        print(f"Train size: {len(train_set)} | Valid size: {len(valid_set)} | Test size: {len(test_set)}")

        # Copy files to their respective destinations
        for source_path in train_set:
            dest_path = os.path.join(dest_dir, 'train', class_name, os.path.basename(source_path))
            shutil.copy(source_path, dest_path)

        for source_path in valid_set:
            dest_path = os.path.join(dest_dir, 'valid', class_name, os.path.basename(source_path))
            shutil.copy(source_path, dest_path)

        for source_path in test_set:
            dest_path = os.path.join(dest_dir, 'test', class_name, os.path.basename(source_path))
            shutil.copy(source_path, dest_path)

    print("\nDataset preparation complete.")

# Paths
SOURCE_DATASET_DIR = "/kaggle/input/20k-multi-class-crop-disease-images"
DESTINATION_DATASET_DIR = "/kaggle/working/new_yolov8_dataset"

# Run the script
if __name__ == '__main__':
    split_dataset(SOURCE_DATASET_DIR, DESTINATION_DATASET_DIR)


Starting dataset preparation...
Source Directory: /kaggle/input/20k-multi-class-crop-disease-images
Destination Directory: /kaggle/working/new_yolov8_dataset
Found 44 classes: ['American Bollworm on Cotton', 'Anthracnose on Cotton', 'Army worm', 'Bacterial Blight in cotton', 'Becterial Blight in Rice', 'Brownspot', 'Common_Rust', 'Cotton Aphid', 'Flag Smut', 'Gray_Leaf_Spot', 'Healthy Maize', 'Healthy Wheat', 'Healthy cotton', 'Leaf Curl', 'Leaf smut', 'Mosaic sugarcane', 'RedRot sugarcane', 'RedRust sugarcane', 'Rice Blast', 'Sugarcane Healthy', 'Tungro', 'Wheat Brown leaf Rust', 'Wheat Brown leaf rust', 'Wheat Stem fly', 'Wheat aphid', 'Wheat black rust', 'Wheat leaf blight', 'Wheat mite', 'Wheat powdery mildew', 'Wheat scab', 'Wheat___Yellow_Rust', 'Wilt', 'Yellow Rust Sugarcane', 'bacterial_blight in Cotton', 'bollrot on Cotton', 'bollworm on Cotton', 'cotton mealy bug', 'cotton whitefly', 'maize ear rot', 'maize fall armyworm', 'maize stem borer', 'pink bollworm in cotton', 'red c

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninst

In [10]:
from ultralytics import YOLO

model = YOLO("yolov8s-cls.pt")

results = model.train(
    data="/kaggle/working/new_yolov8_dataset",
    epochs=100,
    imgsz=224,
    batch=32,
    device=0
)

Ultralytics 8.3.191 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/new_yolov8_dataset, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, p

libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50


val: Scanning /kaggle/working/new_yolov8_dataset/valid... 4901 images, 0 corrupt: 100% ━━━━━━━━━━━━ 4901/4901 2616.9it/s 1.9s0.5s
val: /kaggle/working/new_yolov8_dataset/valid/Wheat black rust/94FX3I.jpg: corrupt JPEG restored and saved
val: New cache created: /kaggle/working/new_yolov8_dataset/valid.cache
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 4 dataloader workers
Logging results to runs/classify/train4
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size
      1/100     0.887G       3.53         32        224: 20% ━━────────── 96/476 11.1it/s 20.1s<34.2s

libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50


      1/100     0.887G      3.024         32        224: 41% ━━━━╸─────── 194/476 11.3it/s 38.5s<25.0s

...

               classes   top1_acc   top5_acc: 27% ━━━───────── 21/77 3.6it/s 6.8s<15.7s

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: cHRM chunk does not match sRGB
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: cHRM chunk does not match sRGB


               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 77/77 3.6it/s 21.6s0.6s


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: cHRM chunk does not match sRGB
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: cHRM chunk does not match sRGB
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all      0.992      0.998
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train4


In [5]:
import os
from ultralytics import YOLO
import pandas as pd

# Load trained model
model = YOLO("/kaggle/input/20kbest/best.pt")

# Validation dataset path
val_root = "/kaggle/working/new_yolov8_dataset/valid"

# Class names
class_names = model.names

# Overall counters
overall_correct, overall_total = 0, 0
class_results = []

# Loop through each class folder inside validation
for true_label in os.listdir(val_root):
    class_folder = os.path.join(val_root, true_label)
    if not os.path.isdir(class_folder):
        continue

    correct, total = 0, 0
    for file in os.listdir(class_folder):
        if file.lower().endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(class_folder, file)

            # Prediction
            results = model.predict(source=img_path, save=False, verbose=False)
            pred_result = results[0]

            if pred_result.probs is not None:
                class_id = int(pred_result.probs.top1)
                pred_label = class_names[class_id]
            else:
                pred_label = "No prediction"

            if pred_label.lower() == true_label.lower():
                correct += 1
            total += 1

    acc = (correct / total) * 100 if total > 0 else 0
    
    # Append the results for the current class to the list
    class_results.append({
        "Class": true_label,
        "Correct": correct,
        "Total": total,
        "Accuracy (%)": round(acc, 2)
    })
    
    overall_correct += correct
    overall_total += total

# Convert the list of results to a DataFrame
df = pd.DataFrame(class_results)

print("\n📊 Per-class Accuracy:")
print(df)

overall_acc = (overall_correct / overall_total) * 100 if overall_total > 0 else 0
print(f"\n Overall Accuracy: {overall_acc:.2f}% ({overall_correct}/{overall_total})")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50



📊 Per-class Accuracy:
                          Class  Correct  Total  Accuracy (%)
0       pink bollworm in cotton        7      8         87.50
1           maize fall armyworm       26     26        100.00
2                 Healthy Maize      209    209        100.00
3                   Wheat aphid       29     29        100.00
4                   Common_Rust      198    198        100.00
5                Wheat Stem fly       34     34        100.00
6   American Bollworm on Cotton        9     11         81.82
7             thirps on  cotton        8      8        100.00
8              RedRot sugarcane      110    110        100.00
9              Wheat black rust       32     33         96.97
10                   Rice Blast      220    220        100.00
11               Healthy cotton       90     90        100.00
12            Sugarcane Healthy       88     90         97.78
13                Healthy Wheat       31     31        100.00
14        Wheat Brown leaf rust        1      2

In [6]:
import os
from ultralytics import YOLO
import pandas as pd

# Load trained model
model = YOLO("/kaggle/input/20kbest/best.pt")

# Validation dataset path
val_root = "/kaggle/working/new_yolov8_dataset/test"

# Class names
class_names = model.names

# Overall counters
overall_correct, overall_total = 0, 0
class_results = []

# Loop through each class folder inside validation
for true_label in os.listdir(val_root):
    class_folder = os.path.join(val_root, true_label)
    if not os.path.isdir(class_folder):
        continue

    correct, total = 0, 0
    for file in os.listdir(class_folder):
        if file.lower().endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(class_folder, file)

            # Prediction
            results = model.predict(source=img_path, save=False, verbose=False)
            pred_result = results[0]

            if pred_result.probs is not None:
                class_id = int(pred_result.probs.top1)
                pred_label = class_names[class_id]
            else:
                pred_label = "No prediction"

            if pred_label.lower() == true_label.lower():
                correct += 1
            total += 1

    acc = (correct / total) * 100 if total > 0 else 0
    
    # Append the results for the current class to the list
    class_results.append({
        "Class": true_label,
        "Correct": correct,
        "Total": total,
        "Accuracy (%)": round(acc, 2)
    })
    
    overall_correct += correct
    overall_total += total

# Convert the list of results to a DataFrame
df = pd.DataFrame(class_results)

print("\n📊 Per-class Accuracy:")
print(df)

overall_acc = (overall_correct / overall_total) * 100 if overall_total > 0 else 0
print(f"\n Overall Accuracy: {overall_acc:.2f}% ({overall_correct}/{overall_total})")

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: cHRM chunk does not match sRGB
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50



📊 Per-class Accuracy:
                          Class  Correct  Total  Accuracy (%)
0       pink bollworm in cotton        8      9         88.89
1           maize fall armyworm       29     29        100.00
2                 Healthy Maize      211    211        100.00
3                   Wheat aphid       30     30        100.00
4                   Common_Rust      200    200        100.00
5                Wheat Stem fly       36     36        100.00
6   American Bollworm on Cotton       12     13         92.31
7             thirps on  cotton       11     11        100.00
8              RedRot sugarcane      112    112        100.00
9              Wheat black rust       36     36        100.00
10                   Rice Blast      222    222        100.00
11               Healthy cotton       91     91        100.00
12            Sugarcane Healthy       89     91         97.80
13                Healthy Wheat       32     33         96.97
14        Wheat Brown leaf rust        1      3

In [5]:
import os
import random
from ultralytics import YOLO
import pandas as pd

# Load trained model
model = YOLO("/kaggle/input/20kbest/best.pt")

# Validation dataset path
val_root = "/kaggle/working/new_yolov8_dataset/test"

# Class names
class_names = model.names

# Overall counters
overall_correct, overall_total = 0, 0
class_results = []

# Loop through each class folder inside validation
for true_label in os.listdir(val_root):
    class_folder = os.path.join(val_root, true_label)
    if not os.path.isdir(class_folder):
        continue

    # Collect only image files
    img_files = [f for f in os.listdir(class_folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

    # Skip if fewer than 4 images
    if len(img_files) < 4:
        continue

    # Pick 5 random images
    selected_imgs = random.sample(img_files, 4)

    correct, total = 0, 0
    for file in selected_imgs:
        img_path = os.path.join(class_folder, file)

        # Prediction
        results = model.predict(source=img_path, save=False, verbose=False)
        pred_result = results[0]

        if pred_result.probs is not None:
            class_id = int(pred_result.probs.top1)
            pred_label = class_names[class_id]
        else:
            pred_label = "No prediction"

        if pred_label.lower() == true_label.lower():
            correct += 1
        total += 1

    acc = (correct / total) * 100 if total > 0 else 0
    
    # Append the results for the current class to the list
    class_results.append({
        "Class": true_label,
        "Correct": correct,
        "Total": total,
        "Accuracy (%)": round(acc, 2)
    })
    
    overall_correct += correct
    overall_total += total

# Convert the list of results to a DataFrame
df = pd.DataFrame(class_results)

print("\n📊 Per-class Accuracy (4 random samples per class):")
print(df)

overall_acc = (overall_correct / overall_total) * 100 if overall_total > 0 else 0
print(f"\n Overall Accuracy: {overall_acc:.2f}% ({overall_correct}/{overall_total})")



📊 Per-class Accuracy (4 random samples per class):
                          Class  Correct  Total  Accuracy (%)
0      Becterial Blight in Rice        4      4         100.0
1             thirps on  cotton        4      4         100.0
2              Wheat black rust        4      4         100.0
3             Wheat leaf blight        4      4         100.0
4                          Wilt        4      4         100.0
5                 Healthy Wheat        4      4         100.0
6            bollworm on Cotton        3      4          75.0
7                red cotton bug        4      4         100.0
8              maize stem borer        4      4         100.0
9                 maize ear rot        4      4         100.0
10            Sugarcane Healthy        4      4         100.0
11        Wheat Brown leaf Rust        4      4         100.0
12         Wheat powdery mildew        4      4         100.0
13               Gray_Leaf_Spot        4      4         100.0
14          Wheat_

In [6]:
import os
import random
from ultralytics import YOLO
import pandas as pd

# Load trained model
model = YOLO("/kaggle/input/20kbest/best.pt")

# Validation dataset path
val_root = "/kaggle/working/new_yolov8_dataset/valid"

# Class names
class_names = model.names

# Overall counters
overall_correct, overall_total = 0, 0
class_results = []

# Loop through each class folder inside validation
for true_label in os.listdir(val_root):
    class_folder = os.path.join(val_root, true_label)
    if not os.path.isdir(class_folder):
        continue

    # Collect only image files
    img_files = [f for f in os.listdir(class_folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

    # Skip if fewer than 4 images
    if len(img_files) < 4:
        continue

    # Pick 5 random images
    selected_imgs = random.sample(img_files, 4)

    correct, total = 0, 0
    for file in selected_imgs:
        img_path = os.path.join(class_folder, file)

        # Prediction
        results = model.predict(source=img_path, save=False, verbose=False)
        pred_result = results[0]

        if pred_result.probs is not None:
            class_id = int(pred_result.probs.top1)
            pred_label = class_names[class_id]
        else:
            pred_label = "No prediction"

        if pred_label.lower() == true_label.lower():
            correct += 1
        total += 1

    acc = (correct / total) * 100 if total > 0 else 0
    
    # Append the results for the current class to the list
    class_results.append({
        "Class": true_label,
        "Correct": correct,
        "Total": total,
        "Accuracy (%)": round(acc, 2)
    })
    
    overall_correct += correct
    overall_total += total

# Convert the list of results to a DataFrame
df = pd.DataFrame(class_results)

print("\n📊 Per-class Accuracy (4 random samples per class):")
print(df)

overall_acc = (overall_correct / overall_total) * 100 if overall_total > 0 else 0
print(f"\n Overall Accuracy: {overall_acc:.2f}% ({overall_correct}/{overall_total})")



📊 Per-class Accuracy (4 random samples per class):
                          Class  Correct  Total  Accuracy (%)
0      Becterial Blight in Rice        4      4         100.0
1             thirps on  cotton        4      4         100.0
2              Wheat black rust        4      4         100.0
3             Wheat leaf blight        4      4         100.0
4                          Wilt        4      4         100.0
5                 Healthy Wheat        4      4         100.0
6                red cotton bug        4      4         100.0
7              maize stem borer        4      4         100.0
8                 maize ear rot        4      4         100.0
9             Sugarcane Healthy        4      4         100.0
10        Wheat Brown leaf Rust        4      4         100.0
11         Wheat powdery mildew        4      4         100.0
12               Gray_Leaf_Spot        4      4         100.0
13          Wheat___Yellow_Rust        4      4         100.0
14                